In [1]:
import numpy as np
import os
import scipy.io as sio
import gc

In [2]:
eeg = np.load("trials/SEED_time_series_25_single_trail.npy")

In [3]:
eeg.shape

(15, 3, 15, 266, 62, 25)

In [4]:
eeg_sub_list = []
for i in range(15):
    eeg_mat_list = []
    for j in range(3):
        eeg_trial_list = []
        for k in range(15):
            zero_index = []
            for m in range(266):
                if(eeg[i][j][k][m].sum() == 0):
                    zero_index.append(m)
            eeg_trial_list.append(np.delete(eeg[i][j][k], zero_index, axis=0))
        eeg_mat_list.append(eeg_trial_list)
    eeg_sub_list.append(eeg_mat_list)

In [5]:
for i in range(15):
    os.mkdir("trials/temporal/subject_" + str(i))
    for j in range(3):
        os.mkdir("trials/temporal/subject_" + str(i) + "/section_" + str(j))

In [6]:
eeg_sub_list[0][0][2].shape

(206, 62, 25)

In [7]:
timeLength = 25

# 电极对应到二维矩阵中，可对应map.xlsx中的图看看
for i in range(15):
    for j in range(3):
        for k in range(15):
            nSample = eeg_sub_list[i][j][k].shape[0]
            eeg_headMap = np.zeros([nSample, 9, 9, timeLength])
            for x in range(nSample):  # 样本数循环
                # 0-1行
                eeg_headMap[x][0][3] = eeg_sub_list[i][j][k][x][0]
                eeg_headMap[x][0][4] = eeg_sub_list[i][j][k][x][1]
                eeg_headMap[x][0][5] = eeg_sub_list[i][j][k][x][2]
                eeg_headMap[x][1][3] = eeg_sub_list[i][j][k][x][3]
                eeg_headMap[x][1][5] = eeg_sub_list[i][j][k][x][4]

                # 2-5行
                for m in range(2, 7):
                    for n in range(9):
                        eeg_headMap[x][m][n] = eeg_sub_list[i][j][k][x][(m - 2) * 9 + n + 5]

                # 第7行
                for m in range(1, 8):
                    eeg_headMap[x][7][m] = eeg_sub_list[i][j][k][x][m + 49]

                # 第8行
                for m in range(2, 7):
                    eeg_headMap[x][8][m] = eeg_sub_list[i][j][k][x][m + 55]
            np.save("trials/temporal/subject_" + str(i) + "/section_" + str(j) + "/trial_temp" + str(k), eeg_headMap)

In [8]:
eeg_headMap.shape

(206, 9, 9, 25)

In [9]:
import matplotlib
import matplotlib.pyplot as plt
import random

freqFirst = eeg_headMap.swapaxes(-1, -2).swapaxes(-2, -3)

randTr = random.randint(1, eeg_headMap.shape[1])
randTime = random.randint(0, 24)

arr = freqFirst[2][0]

plt.matshow(arr, cmap='hot_r')
plt.colorbar()
plt.show()

del freqFirst

<Figure size 480x480 with 2 Axes>

In [10]:
# 设置新的边长
sideLen = 32

In [8]:
del eeg_headMap
gc.collect()

0

In [ ]:
from scipy import interpolate

for i in range(15):
    for j in range(3):
        for k in range(15):
            eeg_headMap = np.load("trials/temporal/subject_" + str(i) + "/section_" + str(j) + "/trial_temp" + str(k) + ".npy")
            eeg_timeFirst = eeg_headMap.swapaxes(-1, -2).swapaxes(-2, -3)
            eegMap_bigger = np.zeros([eeg_timeFirst.shape[0], 25, sideLen, sideLen])
            for m in range(eeg_timeFirst.shape[0]):  # 样本数循环
                for n in range(25):  # 频率循环
                    x = np.arange(9)
                    y = np.arange(9)
                    z = eeg_timeFirst[m][n]  # 取出每一个平面

                    f = interpolate.interp2d(x, y, z, kind='cubic')  # cubic为三次
                    xnew = np.linspace(0, 8, sideLen)
                    ynew = np.linspace(0, 8, sideLen)
                    eegMap_bigger[m][n] = f(xnew, ynew)
            np.save("trials/temporal/subject_" + str(i) + "/section_" + str(j) + "/trial_interpolated_" + str(k) + ".npy", eegMap_bigger)

In [ ]:
# 标准化
#sample * time * 32 * 32
for i in range(15):
    for j in range(3):
        for k in range(15):
            eegMap = np.load("trials/temporal/subject_" + str(i) + "/section_" + str(j) + "/trial_interpolated_" + str(k) + ".npy")
            for m in range(eegMap.shape[0]):
                for n in range(eegMap.shape[1]):
                    eegMap[m][n] = eegMap[m][n] - eegMap[m][n].mean()
                        
            for m in range(eegMap.shape[0]):
                for n in range(eegMap.shape[1]):
                    eegMap[m][n] = eegMap[m][n] / eegMap[m][n].std()

            # 换轴
            eegMap = eegMap.swapaxes(1, 2).swapaxes(2, 3)
    
            #添加维度
            raw_shape = list(eegMap.shape)
            raw_shape.append(1)

            eegMap = eegMap.reshape(raw_shape)
    
            # 降低精度为单精度（float32）
            dt = np.dtype(np.float32)
            eegMap = np.array(eegMap, dtype=dt)
            np.save("trials/temporal/subject_" + str(i) + "/section_" + str(j) + "/trial_" + str(k) + ".npy", eegMap)

In [ ]:
eegMap.shape

In [ ]:
temMap = eegMap.swapaxes(3, 4).swapaxes(2, 3)

import pylab as pl
import matplotlib as mpl

%matplotlib inline
pl.subplot(121)
im1 = pl.imshow(eeg_timeFirst[0][0][0], extent=[-1, 1, -1, 1],
                cmap=mpl.cm.hot, interpolation='nearest', origin="lower")

pl.subplot(122)
im2 = pl.imshow(temMap[0][0][4], extent=[-1, 1, -1, 1],
                cmap=mpl.cm.hot, interpolation='nearest', origin="lower")
pl.colorbar(im2)

pl.show()

del temMap